In [1]:
import numpy as np
%pylab inline
import sympy
from sympy import *
from scipy.optimize import minimize, rosen, rosen_der
## using scipy
from scipy.optimize import fsolve
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
## global parameters
dt,PIE = symbols('dt PIE')

## known values of the structural variables (previous time step)
phif_t,a0x_t,a0y_t,kappa_t,lamdaP_a_t,lamdaP_s_t  = symbols('phif_t a0x_t a0y_t kappa_t lamdaP_a_t lamdaP_s_t')

## structural unknowns
phif,a0x,a0y,kappa,lamdaP_a,lamdaP_s  = symbols('phif a0x a0y kappa lamdaP_a lamdaP_s')

## other variables 
s0x = -a0y
s0y = a0x
#s0x, s0y =  symbols('s0x s0y')

C00, C01, C11 =  symbols('C00 C01 C11')
C = np.array([[C00,C01],[C01,C11]])

In [18]:
### In terms of rotated coordinate system 

#Ce_aa = (1./(lamdaP_a*lamdaP_a))*(C00*a0x*a0x+2*C01*a0x*a0y+C11*a0y*a0y);
#Ce_as = (1./(lamdaP_a*lamdaP_s))*(C00*a0x*s0x+C01*a0x*s0y+C01*s0x*a0y+C11*a0y*s0y);
#Ce_ss = (1./(lamdaP_s*lamdaP_s))*(C00*s0x*s0x+2*C01*s0x*s0y+C11*s0y*s0y);

Ce_aa, Ce_as, Ce_ss =  symbols('Ce_aa Ce_as Ce_ss')

#lamda1 = ((Ce_aa + Ce_ss) + sqrt( (Ce_aa-Ce_ss)*(Ce_aa-Ce_ss) + 4*Ce_as*Ce_as))/2. 
#lamda0 = ((Ce_aa + Ce_ss) - sqrt( (Ce_aa-Ce_ss)*(Ce_aa-Ce_ss) + 4*Ce_as*Ce_as))/2.

lamda1 =  symbols('lamda1')

sinvartheta = (lamda1-Ce_aa)/sqrt(Ce_as*Ce_as + (lamda1-Ce_aa)*(lamda1-Ce_aa));


In [19]:
diff(sinvartheta,lamda1)

(-Ce_aa + lamda1)*(Ce_aa - lamda1)/(Ce_as**2 + (-Ce_aa + lamda1)**2)**(3/2) + 1/sqrt(Ce_as**2 + (-Ce_aa + lamda1)**2)

In [21]:
# given the guess of the growth multipliers at this time step get the new Ce 
a0 = np.array([a0x,a0y])
s0 = np.array([s0x,s0y])
Fginv = (1./lamdaP_a)*np.outer(a0,a0)+(1./lamdaP_s)*np.outer(s0,s0)
Ce = np.dot(Fginv,np.dot(C,Fginv))
print(simplify(Ce[0,0]))

# Change the basis 
Q = np.array([[a0x,s0x],[a0y,s0y]])
Ce_a = np.dot(Q.transpose(),np.dot(Ce,Q))
#print(Ce_a.shape)


1.0*(-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C11*a0x*a0y*(lamdaP_a - lamdaP_s)) + (C00*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C01*a0x*a0y*(lamdaP_a - lamdaP_s))*(a0x**2*lamdaP_s + a0y**2*lamdaP_a))/(lamdaP_a**2*lamdaP_s**2)
(2, 2)


In [22]:
Ce00, Ce01, Ce11 =  symbols('Ce00 Ce01 Ce11')
Tr = Ce00 + Ce11
Det = Ce00*Ce11 - Ce01**2
#Tr = Ce[0,0] + Ce[1,1]
#Det = Ce[0,0]*Ce[1,1] - Ce[0,1]**2
#Tr, Det =  symbols('Tr Det')
lamda1 = Tr/2. + sqrt(Tr**2/4-Det)
lamda0 = Tr/2. - sqrt(Tr**2/4-Det)
#simplify(diff(lamda0,Ce11))

In [37]:
e1 = np.array([Ce11-lamda1,-Ce01])
e1 = e1/sqrt((Ce11-lamda1)**2+Ce01**2)
#e1 = np.array([-Ce01,Ce00-lamda1])
#e1 = e1/(Ce00-lamda1)
#print(simplify(e1[0]))
#print(simplify(e1[1]))


In [38]:
#e1x, e1y =  symbols('e1x e1y')
#e1_ext = np.array([e1x,e1y,0])
e1_ext = np.array([e1[0],e1[1],0])
a0_ext = np.array([a0x,a0y,0])
e1crossa0 = np.cross(e1_ext,a0_ext)
print(simplify(e1crossa0[2]))


(2*Ce01*a0x - a0y*(Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2)))/sqrt(4*Ce01**2 + (Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2))**2)


In [43]:
#print(lamda1)
#diff(lamda1,Ce11)
sinvartheta = e1crossa0[2]
simplify(diff(sinvartheta,Ce11))


(2*a0y*(4*Ce01**2 + (Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2))**2)*(2*Ce00 - 2*Ce11 + 2.0*sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2)) + (2*Ce01*a0x - a0y*(Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2)))*(Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2))*(4*Ce00 - 4*Ce11 + 4.0*sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2)))/(4*(4*Ce01**2 + (Ce00 - 1.0*Ce11 + sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2))**2)**(3/2)*sqrt(-4*Ce00*Ce11 + 4*Ce01**2 + (Ce00 + Ce11)**2))

In [39]:
simplify(diff(Ce[1,1],a0x))

2.0*C00*a0x*a0y**2/lamdaP_s**2 - 4.0*C00*a0x*a0y**2/(lamdaP_a*lamdaP_s) + 2.0*C00*a0x*a0y**2/lamdaP_a**2 - 6.0*C01*a0x**2*a0y/lamdaP_s**2 + 6.0*C01*a0x**2*a0y/(lamdaP_a*lamdaP_s) - 2.0*C01*a0y**3/(lamdaP_a*lamdaP_s) + 2.0*C01*a0y**3/lamdaP_a**2 + 4.0*C11*a0x**3/lamdaP_s**2 + 4.0*C11*a0x*a0y**2/(lamdaP_a*lamdaP_s)

In [19]:
# check that the eigenvectors of this matrix are 
T = Ce[0,0] + Ce[1,1]
D = Ce[0,0]*Ce[1,1] - Ce[0,1]**2
print(simplify(T))
print(simplify(D))

1.0*(-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C11*a0x*a0y*(lamdaP_a - lamdaP_s)) + a0x*a0y*(lamdaP_a - lamdaP_s)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) + (C00*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C01*a0x*a0y*(lamdaP_a - lamdaP_s))*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - (a0x**2*lamdaP_a + a0y**2*lamdaP_s)*(C01*a0x*a0y*(lamdaP_a - lamdaP_s) - C11*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)))/(lamdaP_a**2*lamdaP_s**2)
1.0*(1.0*C00*C11*a0x**8 + 4.0*C00*C11*a0x**6*a0y**2 + 6.0*C00*C11*a0x**4*a0y**4 + 4.0*C00*C11*a0x**2*a0y**6 + 1.0*C00*C11*a0y**8 - 1.0*C01**2*a0x**8 - 4.0*C01**2*a0x**6*a0y**2 - 6.0*C01**2*a0x**4*a0y**4 - 4.0*C01**2*a0x**2*a0y**6 - 1.0*C01**2*a0y**8)/(lamdaP_a**2*lamdaP_s**2)


In [22]:
lam1 = T/2. + sqrt(T**2/4-D)
print(simplify(lam1))

(-1.0*a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C11*a0x*a0y*(lamdaP_a - lamdaP_s)) + 1.0*a0x*a0y*(lamdaP_a - lamdaP_s)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) + lamdaP_a**2*lamdaP_s**2*sqrt((-4.0*(-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C11*a0x*a0y*(lamdaP_a - lamdaP_s)) + (C00*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C01*a0x*a0y*(lamdaP_a - lamdaP_s))*(a0x**2*lamdaP_s + a0y**2*lamdaP_a))*(a0x*a0y*(lamdaP_a - lamdaP_s)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) - (a0x**2*lamdaP_a + a0y**2*lamdaP_s)*(C01*a0x*a0y*(lamdaP_a - lamdaP_s) - C11*(a0x**2*lamdaP_a + a0y**2*lamdaP_s))) + 4.0*(-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*a0x*a0y*(lamdaP_a - lamdaP_s) - C11*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) + (a0x**2*lamdaP_s + a0y**2*lamdaP_a)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)))**2 + 1.0*(-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0

In [37]:
# CHECK against some numerical values
C = np.random.rand(2,2)
C[0,1] = C[0,1]-0.5
C[0,0] = C[0,0]+1
C[1,0] = C[0,1]
C[1,1] = C[1,1]+1
C00 = C[0,0]
C01 = C[0,1]
C11 = C[1,1]
#F = np.random.rand(2,2)
#C = np.dot(F.transpose(),F)
#C = np.array([[2.,0],[0.,2.]])
lamdaP_a = 0.8
lamdaP_s = 0.9
a0x = 0.4
a0y = np.sqrt(1-a0x**2)
s0x = -a0y
s0y = a0x
Ce_v1 = np.zeros((2,2))
Ce_v1[0,0] = (-a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C11*a0x*a0y*(lamdaP_a - lamdaP_s)) + (C00*(a0x**2*lamdaP_s + a0y**2*lamdaP_a) - C01*a0x*a0y*(lamdaP_a - lamdaP_s))*(a0x**2*lamdaP_s + a0y**2*lamdaP_a))/(lamdaP_a**2*lamdaP_s**2)
Ce_v1[0,1] = (a0x*a0y*(lamdaP_a - lamdaP_s)*(C01*a0x*a0y*(lamdaP_a - lamdaP_s) - C11*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) - (a0x**2*lamdaP_s + a0y**2*lamdaP_a)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)))/(lamdaP_a**2*lamdaP_s**2)
Ce_v1[1,1] = (a0x*a0y*(lamdaP_a - lamdaP_s)*(C00*a0x*a0y*(lamdaP_a - lamdaP_s) - C01*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)) - (a0x**2*lamdaP_a + a0y**2*lamdaP_s)*(C01*a0x*a0y*(lamdaP_a - lamdaP_s) - C11*(a0x**2*lamdaP_a + a0y**2*lamdaP_s)))/(lamdaP_a**2*lamdaP_s**2)
Ce_v1[1,0] = Ce_v1[0,1]

## the second way is with my formula in the current code 
Ce_aa = (1./(lamdaP_a*lamdaP_a))*(C00*a0x*a0x+2*C01*a0x*a0y+C11*a0y*a0y);
Ce_as = (1./(lamdaP_a*lamdaP_s))*(C00*a0x*s0x+C01*a0x*s0y+C01*s0x*a0y+C11*a0y*s0y);
Ce_ss = (1./(lamdaP_s*lamdaP_s))*(C00*s0x*s0x+2*C01*s0x*s0y+C11*s0y*s0y);
Ce_rot = np.array([[Ce_aa,Ce_as],[Ce_as,Ce_ss]])
Q = np.array([[a0x,s0x],[a0y,s0y]])
Ce_v2 = np.dot(Q,np.dot(Ce_rot,Q.transpose()))
         
a0 = np.array([a0x,a0y])
s0 = np.array([s0x,s0y])
Fg = lamdaP_a*np.outer(a0,a0) + lamdaP_s*np.outer(s0,s0)
Fginv = np.linalg.inv(Fg)
Ce_v3 = np.dot(Fginv,np.dot(C,Fginv))

print('Ce from formula')
print(Ce_v1)
print('Ce from rotated coordinates')
print(Ce_v2)
print('Ce from Fginv and C')
print(Ce_v3)
print('\n')

w,v = np.linalg.eig(Ce_v1)
print(w)
print(v)

# alternative with the calculation above 
T = Ce_v1[0,0]+Ce_v1[1,1]
D = Ce_v1[0,0]*Ce_v1[1,1] - Ce_v1[0,1]**2
lam1 = T/2. + np.sqrt(T**2/4 - D)
lam2 = T/2. - np.sqrt(T**2/4 - D)
if np.abs(Ce_v1[0,1])<1e-7 and np.abs(lam2-lam1)<1e-7:
    e1 = np.array([1,0])
    e2 = np.array([0,1])
elif np.abs(Ce_v1[0,1])<1e-7 and np.abs(lam2-lam1)>1e-7:
    if np.abs(Ce_v1[0,0]-lam1)>1e-7:
        e1 = np.array([-Ce_v1[0,1],Ce_v1[0,0]-lam1])
        e1 = e1/np.linalg.norm(e1)
        e2 = np.array([-e1[1],e1[0]])
    elif np.abs(Ce_v1[1,1]-lam1)>1e-7:
        e1 = np.array([Ce_v1[1,1]-lam1,-Ce_v1[0,1]])
        e1 = e1/np.linalg.norm(e1)
        e2 = np.array([-e1[1],e1[0]])
    else:
        print('something wrong, Ce_v1=0, lam2!=lam1, but both a-lam1, d-lam1 = 0')
else:
    e1 = np.array([Ce_v1[1,1]-lam1,-Ce_v1[0,1]])
    e1 = e1/np.linalg.norm(e1)
    e2 = np.array([-e1[1],e1[0]])

print('lami, ei')        
print(lam1,lam2)
print(e1,e2)
sinvartheta_v1 = (-2*Ce_v1[0,1]*a0x + a0y*(-2*Ce_v1[1,1] + T + np.sqrt(-4*D + T*T)))/sqrt(4*Ce_v1[0,1]*Ce_v1[0,1] + (-2*Ce_v1[1,1] + T + sqrt(-4*D + T*T))**2);
if(np.abs(lam1-lam2)<1e-8 or np.abs(lam1-Ce_aa)<1e-8):
    # equal eigenvalues means multiple of identity -> you can't possibly reorient. 
    # or, eigenvector in the direction of a0 already -> no need to reorient since you are already there
    sinvartheta_v2 = 0.
else:
    # if eigenvalues are not the same and the principal eigenvalue is not already in the direction of a0
    sinvartheta_v2 = (lam1-Ce_aa)/sqrt(Ce_as*Ce_as + (lam1-Ce_aa)*(lam1-Ce_aa));

e1_ext = np.array([e1[0],e1[1],0])
a0_ext = np.array([a0[0],a0[1],0])
sinvartheta = np.cross(e1,a0)
sinvartheta = np.linalg.norm(sinvartheta)
print('sinvartheta: v1 (from formula), v2 (from rotated system), v3 (from cross product)')
print(sinvartheta_v1)
print(sinvartheta_v2)
print(sinvartheta)

Ce from formula
[[ 2.49247418  0.63784497]
 [ 0.63784497  1.844225  ]]
Ce from rotated coordinates
[[ 2.49247418  0.63784497]
 [ 0.63784497  1.844225  ]]
Ce from Fginv and C
[[ 2.49247418  0.63784497]
 [ 0.63784497  1.844225  ]]


[ 2.88382352  1.45287566]
[[ 0.8523558  -0.52296232]
 [ 0.52296232  0.8523558 ]]
lami, ei
(2.8838235220253696, 1.4528756552513209)
(array([-0.8523558 , -0.52296232]), array([ 0.52296232, -0.8523558 ]))
sinvartheta: v1 (from formula), v2 (from rotated system), v3 (from cross product)
0.572012065375844
0.572012065375844
0.572012065376


In [24]:
# my original code

Ce_aa = (1./(lamdaP_a*lamdaP_a))*(C00*a0x*a0x+2*C01*a0x*a0y+C11*a0y*a0y);
Ce_as = (1./(lamdaP_a*lamdaP_s))*(C00*a0x*s0x+C01*a0x*s0y+C01*s0x*a0y+C11*a0y*s0y);
Ce_ss = (1./(lamdaP_s*lamdaP_s))*(C00*s0x*s0x+2*C01*s0x*s0y+C11*s0y*s0y);
lamda1 = ((Ce_aa + Ce_ss) + np.sqrt( (Ce_aa-Ce_ss)*(Ce_aa-Ce_ss) + 4*Ce_as*Ce_as))/2.; # the eigenvalue is a squared number by notation
lamda0 = ((Ce_aa + Ce_ss) - np.sqrt( (Ce_aa-Ce_ss)*(Ce_aa-Ce_ss) + 4*Ce_as*Ce_as))/2.; # the eigenvalue is a squared number by notation
Ce_v2 = np.zeros((2,2))
Ce_v2[0,0] = Ce_aa
Ce_v2[0,1] = Ce_as
Ce_v2[1,0] = Ce_as
Ce_v2[1,1] = Ce_ss

w,v = np.linalg.eig(Ce_v2)
print(w)
print(v)

print(lamda1)
print(lamda0)

sinvartheta_v3 = (lamda1-Ce_aa)/np.sqrt(Ce_as*Ce_as + (lamda1-Ce_aa)*(lamda1-Ce_aa));
print(sinvartheta_v3)

[ 2.91911078  2.25021057]
[[ 0.99308986  0.11735646]
 [-0.11735646  0.99308986]]
2.91911078168
2.25021057158
0.117356458068


In [31]:
np.linalg.norm(a0)

0.93037626796904049

In [ ]:
// compute with eigen instead
		//FFg = lamdaP(0)*a0xa0 + lamdaP(1)*s0xs0;
		//CCe = Fg*CC*Fg; 
		//lamdae_i, ei = eigen ;
		//sinvartheta = norm(a0*e1);
		